In [2]:
import networkx as nx
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram

import numpy as np
from scipy.optimize import minimize

In [3]:
# dij需求i到设施j的成本
d = [[1, 2], [1, 2]]
n = 2   # 两个设施点
m = 2   # 两个需求点
# d = [[1, 2], [3, 4], [5, 6]]
# n = 2   # 两个设施点
# m = 3   # 三个需求点
num_qubits = n + 2 * n * m

# gi设施i的建设成本
g = [2, 1]


In [4]:
penalty = 40
depth = 5
params = np.ones(depth * 2)

In [5]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

In [6]:
# dtype=np.complex128
def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H


def generate_Hp(n, m, d, g):
    # 初始化 Hp 矩阵为零矩阵
    # print(num_qubits)
    Hp = np.zeros((2**num_qubits, 2**num_qubits))
    for i in range(m):
        for j in range(n):
            Hp += d[i][j] * (add_in_target(num_qubits, n * (1 + i) + j) - np.eye(2**num_qubits)) / 2
    
    for j in range(n):
        Hp +=  g[j] * (add_in_target(num_qubits, j)- np.eye(2**num_qubits)) / 2

    # for i in range(m):
    #     Ht = np.zeros((2**num_qubits, 2**num_qubits))
    #     for j in range(n):
    #         Ht += (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + i) + j)) / 2
    #     Ht -= np.eye(2**num_qubits)
    #     Hp += -penalty * Ht @ Ht
    
    for i in range(m):
        for j in range(n):
            Ht = (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + i) + j)) / 2 + (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + m + i) + j)) / 2 + (np.eye(2**num_qubits) - add_in_target(num_qubits, j)) / 2
            Hp += -penalty * Ht @ Ht

    return Hp

In [7]:
def gnrt_Hd(n):
  Hd = np.zeros((2**n, 2**n)).astype(np.complex128)
  for i in range(n):
    j = (i + 1) % n
    Hd += (add_in_target(n, i, GateX) @ add_in_target(n, j, GateX) + add_in_target(n, i, GateY) @ add_in_target(n, j, GateY))
  return -Hd


In [8]:
from scipy.linalg import expm
def build_circ(n, m, d, g, params):
  qc = QuantumCircuit(num_qubits)
  beta = params[:depth]
  gamma = params[depth:]
  for i in range(m * n):
    qc.h(i)
  for i in range(m * n, m * n * 2, 2):
    qc.x(i)
  for i in range(m * n * 2, num_qubits):
    qc.h(i)
  for dp in range(depth):
    qc.unitary(expm(-1j * gamma[dp] * generate_Hp(n, m, d, g)), range(num_qubits)) # transpile
    # 
    for i in range(m * n):
      qc.rx(beta[dp], i)
    for i in range(m):
      qc.unitary(expm(-1j * beta[dp] * gnrt_Hd(n)), range(n * (m + i) , n * (m + i + 1)))
    for i in range(2 * m * n, num_qubits):
      qc.rx(beta[dp], i)
  qc.measure_all()
  return qc

In [9]:
print(build_circ(n, m, d, g, np.full(depth * 2, np.pi/4)))

         ┌───┐┌──────────┐┌─────────┐ ┌──────────┐┌─────────┐ ┌──────────┐»
    q_0: ┤ H ├┤0         ├┤ Rx(π/4) ├─┤0         ├┤ Rx(π/4) ├─┤0         ├»
         ├───┤│          │├─────────┤ │          │├─────────┤ │          │»
    q_1: ┤ H ├┤1         ├┤ Rx(π/4) ├─┤1         ├┤ Rx(π/4) ├─┤1         ├»
         ├───┤│          │├─────────┤ │          │├─────────┤ │          │»
    q_2: ┤ H ├┤2         ├┤ Rx(π/4) ├─┤2         ├┤ Rx(π/4) ├─┤2         ├»
         ├───┤│          │├─────────┤ │          │├─────────┤ │          │»
    q_3: ┤ H ├┤3         ├┤ Rx(π/4) ├─┤3         ├┤ Rx(π/4) ├─┤3         ├»
         ├───┤│          │├─────────┴┐│          │├─────────┴┐│          │»
    q_4: ┤ X ├┤4         ├┤0         ├┤4         ├┤0         ├┤4         ├»
         └───┘│  Unitary ││  Unitary ││  Unitary ││  Unitary ││  Unitary │»
    q_5: ─────┤5         ├┤1         ├┤5         ├┤1         ├┤5         ├»
         ┌───┐│          │├──────────┤│          │├──────────┤│          │»
    q_6: ┤ X

In [10]:
def cost_function(x):
  num = [int(char) for char in x]
  C = 0
  for i in range(m):
    for j in range(n):
      C += d[i][j] * num[n * (1 + i) + j]
      
  for j in range(n):
    C += g[j] * num[j]

  # for i in range(m):
  #   t = 0
  #   for j in range(n):
  #     t += num[n * (1 + i) + j]
  #   C += penalty * (t - 1)**2

  for i in range(m):
    for j in range(n):
      C += penalty * (num[n * (1 + i) + j] + num[n * (1 + m + i) + j] - num[j]) ** 2
  return C

def compute_expectation(counts):
  EV = 0
  total_count = 0
  for x, count in counts.items():
    C = cost_function(x)
    EV += C*count
    total_count += count

  return EV/total_count


def expectation_from_sample(shots = 2000):
  backend = Aer.get_backend('qasm_simulator')
  backend.shots = shots

  def execute_circ(theta):
    qc = build_circ(n, m, d, g, theta)
    counts = backend.run(qc, seed_simulator=10, shots=shots).result().get_counts()
    return compute_expectation(counts)
  
  return execute_circ

In [11]:
cost_function('1010100000')

4

In [12]:
from numpy.lib.utils import source
from scipy.optimize import minimize
import numpy as np
# 初始化迭代计数器
iteration_count = 0
def test(pen, dep, par):
  global penalty, depth, params, iteration_count
  iteration_count = 0
  penalty = pen
  depth = dep
  params = par
  expectation = expectation_from_sample()
  def callback(x):
      global iteration_count
      iteration_count += 1
      if iteration_count % 10 == 0:
          print(f"Iteration {iteration_count}, Result: {expectation(x)}")
  # 设定最大迭代次数
  max_iterations = 1000

  # 使用 COBYLA 方法进行最小化，并设置 callback 函数
  res = minimize(expectation, params, method='COBYLA', options={'maxiter': max_iterations}, callback=callback)
  # 输出最终结果
  print("Final Result:", res)
  backend = Aer.get_backend('aer_simulator')
  backend.shots = 100000

  shots=100000
  qc_res = build_circ(n, m, d, g, params=res.x)

  counts = backend.run(qc_res, seed_simulator=10, shots = shots).result().get_counts()
  # plot_histogram(counts)
  sorted_counts = sorted(counts, key=counts.get, reverse=True)
  print("\n----------------- Full result ---------------------")
  print("selection\t\tprobability\tvalue")
  print("---------------------------------------------------")
  for x in sorted_counts[:20]:
    print(x, "{:.1f}%".format(counts[x] / shots * 100), cost_function(x))

In [13]:
test(40, 5, np.full(5 * 2, np.pi/5))

Iteration 10, Result: 129.277
Iteration 20, Result: 178.413
Iteration 30, Result: 165.5485
Iteration 40, Result: 164.8365
Iteration 50, Result: 103.5605
Iteration 60, Result: 66.688
Iteration 70, Result: 58.696
Iteration 80, Result: 56.84
Iteration 90, Result: 60.09
Iteration 100, Result: 56.352
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 57.0265
       x: [ 6.313e-01  6.145e-01  1.541e+00  1.869e+00  6.263e-01
            6.218e-01  6.340e-01  6.252e-01  6.369e-01  1.645e+00]
    nfev: 108
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
0101010000 37.0% 5
0001010000 14.1% 84
0101100000 5.2% 84
0110010000 5.2% 84
0101100001 3.2% 44
0110010100 3.1% 44
0101011000 2.4% 45
1101010000 2.3% 87
0101010010 2.3% 45
1101011000 2.1% 47
1101010010 2.1% 47
0001100000 1.7% 83
0010010000 1.6% 83
0110011000 1.0% 204
0101100010 1.0% 204
1001010010 0.

In [14]:
test(40, 3, np.full(3 * 2, np.pi/3))

Iteration 10, Result: 173.822
Iteration 20, Result: 158.1385
Iteration 30, Result: 127.507
Iteration 40, Result: 109.4165
Iteration 50, Result: 110.075
Iteration 60, Result: 110.5175
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 110.5175
       x: [ 2.344e+00  1.087e+00  2.048e+00  1.020e+00  1.001e+00
            1.277e+00]
    nfev: 60
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
0101010101 30.8% 85
1101010101 7.0% 167
0101011101 4.3% 125
0101010111 4.3% 125
0101010000 4.2% 5
0101010100 2.6% 45
0101010001 2.5% 45
0101011111 2.3% 165
1110010101 1.4% 86
1101100100 1.4% 126
1110010001 1.4% 126
1101011111 1.3% 87
1101100101 1.3% 86
0110010101 1.0% 84
0101100101 1.0% 84
0101100100 1.0% 124
1101011101 1.0% 127
1101010111 0.9% 127
0110010001 0.9% 124
1001100100 0.9% 205
